<a href="https://colab.research.google.com/github/hydanggia/woundhealingproject/blob/master/ADIMethodHeatEquation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import numpy as np
def u_exact(x, y,t):
        return math.sin(x)*math.sin(y)*math.e**((-2)*t)  # fulfills BC at x=0 and x=L
def I(x,y):
        return u_exact(x,y,0)
def bc(x,y,t):
        return u_exact(x,y,t)
def f(x,y):
        return math.sin(x)*math.sin(y)


In [0]:
import numpy as np
from scipy import linalg
def solver_ADI(I,f,bc,Lx,Ly,Nx,Ny,dt,T):
  dx = Lx/float(Nx)
  dy = Ly/float(Ny)
  x = np.linspace(0,Lx,Nx+1) #Create points in direction of x
  y = np.linspace(0,Ly,Ny+1) #Create points in direction of y
  u = np.zeros((Nx+1,Ny+1),float) #Create Solution Array.
  u_half = u.copy() #Create a solution at t + dt/2
  u_last = u.copy() #Create a solution at t + dt

  C_x = dt/dx**2 #Constant for equation in x part
  C_y = dt/dy**2 #Constant for equation in y part

  #Set initial condition: 
  t = 0.0
  for i in range(0,Nx+1):
    for j in range(0,Ny+1): 
      u[i,j] = I(x[i],y[j])

  while t<= T:        #Create t<=T
    t_last = t        #Set t_last = last t
    t +=0.5*dt        #Set new t = t + 1/2dt
    #Sweep in x direction
    j = 0 #left boundary
    for i in range(0,Nx+1):
      u_half[i,j] = bc(x[i],y[j],t)
    #Solve tridiagonal system for internal columns j
    for j in range(0,Nx):
      A = np.zeros(shape = (Nx+1,Nx+1))
      b = np.zeros(Nx+1,float)
      #First treat lower boundary for col j
      i = 0
      A[i,i] = 1 #First in diagonal
      b[i] = bc(x[i],y[j],t)
      #Run through inner points for col j
      for i in range(1,Nx):
        A[i,i-1] = -C_x
        A[i,i] = 2 + 2*C_x
        A[i,i+1] = -C_x
        b[i] = 2*u[i,j] + C_y*(u[i,j-1] - 2*u[i,j]+u[i,j+1])
        #Second treat upper boundary
      i = Nx
      A[i,i] = 1
      b[i] = bc(x[i],y[j],t)
        #Solve system x = A^(-1) * b
      temp = linalg.solve(A,b)
        #Insert sol into col j
      for i in range(0,Nx+1):
        u_half[i,j] = temp[i]
    j = Ny #right boundary
    for i in range(0,Nx+1):
      u_half[i,j] = bc(x[i],y[j],t)
    t_last = t
    t +=0.5*dt
    # Sweep in y direction
    i = 0 #lower boundary
    for j in range(0,Ny+1):
      u_last[i,j] = bc(x[i],y[j],t)
    #Solve tridiagonal system for internal row i
    for i in range(1,Nx):
      B = np.zeros(shape = (Ny+1,Ny+1))
      c = np.zeros(Ny+1,float)
      #First treat left boundary for row i
      j = 0
      B[j,j] = 1 #First in diagonal
      c[j] = bc(x[i],y[j],t)
      #Run through inner points for col j
      for j in range(1,Ny):
        B[j,j-1] = -C_y
        B[j,j] = 2 + 2*C_y
        B[j,j+1] = -C_y
        c[j] = 2*u_half[i,j] + C_x*(u_half[i-1,j] - 2*u_half[i,j]+u_half[i+1,j]) 
        #Second treat right boundary
      j = Ny
      B[j,j] = 1
      c[j] = bc(x[i],y[j],t)
        #Solve system x = A^(-1) * b
      temp = linalg.solve(A,c)
        #Insert sol into col j
      for j in range(0,Ny+1):
        u_last[i,j] = temp[j]
    i = Nx
    for j in range(0,Ny+1):
      u_last[i,j] = bc(x[i],y[j],t)   
    u = u_last.copy()   
  return u
